<a href="https://colab.research.google.com/github/LuizFelipe22/desduplicacao_endereco/blob/main/Ol%C3%A1%2C_este_%C3%A9_o_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import dedupe
import pandas as pd
import numpy as np

In [2]:
endereco = pd.read_csv('arquivos/Deduplica_Endereco.csv', encoding='utf-8').replace(np.nan, None)

endereco = endereco.loc[(endereco['rua'].notnull()) & (endereco['bairro'].notnull())]

In [3]:
cep = pd.read_csv('arquivos/cep_tratado.csv', encoding='utf-8').replace(np.nan, None)

cep_util = cep.rename(columns={'localidade': 'municipio', 'logradouroDNEC':'logradouro'})[['cep', 'logradouro', 'bairro', 'municipio']]

In [4]:
iguais = []
diferentes = []

for z, i in endereco.iterrows():
    if i['rua'] == 1.0 and i['bairro'] == 1.0:
        igual = ({'logradouro': i['logradouro_1'], 'bairro': i['bairro_1'], 'municipio': i['municipio_1']},
                 {'logradouro': i['logradouro_2'], 'bairro': i['bairro_2'], 'municipio': i['municipio_2']})
        iguais.append(igual)
    elif i['rua'] == 0.0 or i['bairro'] == 0.0:
        diferente = ({'logradouro': i['logradouro_1'], 'bairro': i['bairro_1'], 'municipio': i['municipio_1']},
                     {'logradouro': i['logradouro_2'], 'bairro': i['bairro_2'], 'municipio': i['municipio_2']})
        diferentes.append(diferente)

labeled_examples = {
    "match": iguais,
    "distinct": diferentes,
}

In [5]:
df_1 = endereco[['logradouro_1', 'bairro_1','municipio_1']]
df_2 = endereco[['logradouro_2', 'bairro_2','municipio_2']]

df_1.columns = ['logradouro', 'bairro','municipio']
df_2.columns = ['logradouro', 'bairro','municipio']
df_1 = df_1.to_dict(orient='index')
df_2 = df_2.to_dict(orient='index')

In [6]:
variables = [
    {'field' : 'logradouro', 'type': 'String', 'has missing':True},
    {'field' : 'bairro', 'type': 'String', 'has missing':True},
    {'field' : 'municipio', 'type': 'String', 'has missing':True}
]


deduper = dedupe.RecordLink(variables)

In [7]:
deduper.mark_pairs(labeled_examples)

In [8]:
with open('training.json', 'w') as f:
    deduper.write_training(f)

In [9]:
with open('training.json', 'r') as file:
  deduper.prepare_training(df_1,df_2,training_file=file)

In [ ]:
deduper.train(recall=0.9)